In [1]:
#! venv/bin/ python3
# *-encoding: utf8 -*

import requests as rqs
from bs4 import BeautifulSoup
import datetime
import time
import pandas as pd


def unixTimeStampConvert(num):
    convert = datetime.datetime.fromtimestamp(int(num)).strftime('%Y-%m-%d %H:%M:%S')
    return convert

In [2]:
# CONEXION AL SERVIDOR (SITIO WEB), Y REQUEST (PETICION).
# **********************************************************
# A que sitio me quiero conectar, parametro y URI
myurl = "https://votaciones.hcdn.gob.ar"
# Los siguientes datos los saque de la misma pagina, mirando el codigo fuente para ver
# como enviaba las peticiones y cual debia enviar para obtener todos los datos =)
formQuery = {'anoSearch': -1, 'txtSearch': ''}
endPoint = "https://votaciones.hcdn.gob.ar/votaciones/search"

# Peticion al servidor
r = rqs.post(endPoint, data=formQuery)

# Estado de la conexion
status = r.status_code

if status == 200:
    print(f'Conectando a {myurl}')
    print(f'Estado de la conexion :: {status}')
else:
    print('La conexion no se establecio correctamente')
    print(f'Codigo estado : {status}')
    quit()
# **********************************************************

Conectando a https://votaciones.hcdn.gob.ar
Estado de la conexion :: 200


In [3]:
# Antes de programar el scraper deberiamos saber bien cuanl es el objeto html que contiene la informacion que
# queremos por ejemplo, cual es el tag que contiene la info que queremos.
# podemos identificar el objeto con la informacion a traves del nombre del Tag, del selctor css, "class", del id, etc

# El elemento que me interesa tiene la clase "row-acta"
clase = "row-acta"

# Creo el objeto BeautifulSoup
soup = BeautifulSoup(r.content, 'html.parser')

# El tipo de dato es un result set de beautiful soup, es un iterable, algo asi como una lista
datos = soup.find_all("tr", class_=clase)

In [4]:
# EXTRACCION DE LAS URLS DONDE ESTAN LOS DETALLES DE CADA VOTACION
# **********************************************************
urlActas = list()
for t in datos:
    # Columnas
    cols = t.find_all('td')
    idActa = t['id']  # Seria Clave principal
    date = unixTimeStampConvert(t['data-date']) # Tengo que convertir el formato de la fecha en human friendly =)
    # Me voy a guardar los datos que extraje en una lista de tuplas
    urlActas.append((myurl + cols[4].find('a')['href'], idActa, date))
# **********************************************************

In [5]:
# RECORRIDO DE LAS ACTAS Y SCRAPING DE LOS DETALLES - ACTAS Y VOTACION
# **********************************************************
actas = list()
votaciones = list()
count = 1
for a in urlActas:
    #Conexion a cada Acta # *********************************
    r = rqs.get(a[0])
    soup = BeautifulSoup(r.content, 'html.parser')
    
    # idActa = a[1] este es el id que figura en el sitio, pero voy a usar otro
    dateActa = a[2]
    urlActa = a[0]

    # VAmos a recuperar estos datos -> Período 123 - Reunión 40 - Acta 31
    text = soup.find('h5').text.split('-')
    periodo = text[0].split()[1]
    reunion = text[1].split()[1]
    numeroActa = text[2].split()[1]
    idSite = a[1]
    idActa = idVotacion = '{}-{}-{}'.format(periodo, reunion, numeroActa)

    # Acta # *********************************
    div = soup.find('div', class_='white-box')
    h3 = div.find_all('h3')
    h4 = div.find_all('h4')

    titulo = h4[0].text.strip().lower()
    presidente = h4[1].find('b').text
    resolucion = h3[0].text

    actas.append({
            'idActa':idActa,
            'periodo': periodo,
            'reunion': reunion,
            'numeroActa': numeroActa,
            'titulo': titulo,
            'presidente': presidente,
            'resolucion': resolucion,
            'urlActa': urlActa
        }
    )

    # Votacion # *********************************
    tabla = soup.find('table', id='myTable')
    rows = tabla.find_all('tr')
#     count = 1
    votos = list()
    for r in rows:
        cols = [x for x in r.find_all('td')]
        if cols:
            try:
                idDip = cols[0].find('div')['id'].split('-')[1]
            except TypeError:
                pass
            
            voto = cols[4].text.strip().lower()
            dichos = cols[5].text.strip().lower()
            
            votos.append({
                    'idVotacion': idVotacion,
                    'idVoto': count,
                    'voto': voto,
                    'numeroActa': numeroActa,
                    'dip': idDip,
                    'dichos': dichos               
            })
            count += 1
            
#     if not votos:
#         continue
#     else:
#         votaciones.append(votos)
        
#     time.sleep(1)
# **********************************************************

KeyboardInterrupt: 

In [ ]:
# votaciones

*******************************

In [6]:
# CONEXION A LA PAGINA DE DONDE VAMOS A SACAR LOS DATOS DE TODOS LOS DIPUTADOS
# Los datos de cada diputado estan en una pagina diferente, por eso voy a hacer nuevas peticiones
# **********************************************************
myurl = "https://votaciones.hcdn.gob.ar/estadisticas/home"
# Los siguientes datos los saque de la misma pagina, mirando el codigo fuente para ver
# como enviaba las peticiones y cual debia enviar para obtener todos los datos =)
formQuery = {'anoSearchEstadistica': -1}
endPoint = "https://votaciones.hcdn.gob.ar/estadisticas/search"

# Peticion al servidor
r = rqs.post(endPoint, data=formQuery)

# Estado de la conexion
status = r.status_code

if status == 200:
    print(f'Conectando a {myurl}')
    print(f'Estado de la conexion :: {status}')
else:
    print('La conexion no se establecio correctamente')
    print(f'Codigo estado : {status}')
    quit()
# **********************************************************

soup = BeautifulSoup(r.content, 'html.parser')
table = soup.find('table', id='myTable')
rows = table.find_all('tr')

# for r in rows:
#     cols = r.find_all('td')
    

Conectando a https://votaciones.hcdn.gob.ar/estadisticas/home
Estado de la conexion :: 200


In [7]:
myurl = 'https://votaciones.hcdn.gob.ar'

In [8]:
diputados = list()
for r in rows:
    cols = r('td')
    if cols:

        idDip = cols[0].find('div')['id'].split('-')[1]
        urlPerf = myurl + cols[1].find('a')['href']
        fullName = cols[1].text.strip()

        if len(fullName) > 0:
            apellido = fullName.split(',')[0]
            nombre = fullName.split(',')[1]
        else:
            apellido = ''
            nombre = ''

        estado = cols[2].text.strip()
        bloque = cols[3].text.strip()
        provincia = cols[4].text.strip()
        afirmativos = cols[5].text.strip()
        negativos = cols[6].text.strip()
        abstenciones = cols[7].text.strip()
        ausencias = cols[8].text.strip()
        presidencias = cols[9].text.strip()

#         diputados.append({
#             idDip: {
#                 'idDip': idDip,
#                 'urlPerf': urlPerf,
#                 'fullName': fullName,
#                 'apellido': apellido,
#                 'nombre': nombre,
#                 'estado': estado,
#                 'bloque': bloque,
#                 'provincia': provincia,
#                 'afirmativos': afirmativos,
#                 'negativos': negativos,
#                 'abstenciones': abstenciones,
#                 'ausencias': ausencias,
#                 'presidencias': presidencias
#                 }
#             }
#         )
        diputados.append({
                    'idDip': idDip,
                    'urlPerf': urlPerf,
                    'fullName': fullName,
                    'apellido': apellido,
                    'nombre': nombre,
                    'estado': estado,
                    'bloque': bloque,
                    'provincia': provincia,
                    'afirmativos': afirmativos,
                    'negativos': negativos,
                    'abstenciones': abstenciones,
                    'ausencias': ausencias,
                    'presidencias': presidencias
                    }
                )     

In [9]:
# # Convertir la lista de diputados en un Json
# import json as js
# dipJson = {'diputados':diputados}
# dipJson = js.dumps(dipJson, indent=4)

In [10]:
# [list(x .keys())[0] for x in diputados]
pd.DataFrame([x for x in diputados])

,idDip,urlPerf,fullName,apellido,nombre,estado,bloque,provincia,afirmativos,negativos,abstenciones,ausencias,presidencias
0,A50768,https://votaciones.hcdn.gob.ar/diputado/1722,". NO INCORPORADO, Legislador 1",. NO INCORPORADO,Legislador 1,-,Justicialista por Tucumán,Tucumán,-,-,-,27,-
1,A50774,https://votaciones.hcdn.gob.ar/diputado/1719,". NO INCORPORADO, Legislador 2",. NO INCORPORADO,Legislador 2,-,PRO,C.A.B.A.,-,-,-,36,-
2,A4068,https://votaciones.hcdn.gob.ar/diputado/596,"ABALOS, Roberto José",ABALOS,Roberto José,-,Unión Cívica Radical,Santiago del Estero,61,96,6,50,-
3,G344,https://votaciones.hcdn.gob.ar/diputado/1531,"ABALOVICH, Eduardo Antonio",ABALOVICH,Eduardo Antonio,-,Unión Cívica Radical,Santiago del Estero,41,33,4,22,-
4,G309,https://votaciones.hcdn.gob.ar/diputado/1496,"ABAN, Gloria del S.",ABAN,Gloria del S.,-,Justicialista,Salta,58,23,2,17,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751,A51101,https://votaciones.hcdn.gob.ar/diputado/1675,"ZOTTOS, Miguel Andrés Costas",ZOTTOS,Miguel Andrés Costas,En actividad,Justicialista,Salta,418,161,52,366,-
1752,A3629,https://votaciones.hcdn.gob.ar/diputado/1176,"ZUCCARDI, Cristina",ZUCCARDI,Cristina,-,FREPASO,Mendoza,62,24,6,73,-
1753,G297,https://votaciones.hcdn.gob.ar/diputado/1481,"ZUCCARDI de FLAMARIQUE, María",ZUCCARDI de FLAMARIQUE,María,-,Justicialista,Mendoza,52,28,1,83,-
1754,A78,https://votaciones.hcdn.gob.ar/diputado/1177,"ZUÑIGA, Ovidio Octavio",ZUÑIGA,Ovidio Octavio,-,Justicialista,Rio Negro,70,61,7,27,-


In [11]:
pd.DataFrame([x for x in actas])

,idActa,periodo,reunion,numeroActa,titulo,presidente,resolucion,urlActa
0,137-12-27,137,12,27,expediente 0016-pe-2019 - o.d. 2 - ley de soli...,"MASSA, Sergio Tomás",AFIRMATIVO,https://votaciones.hcdn.gob.ar/votacion/3959
1,137-12-26,137,12,26,expediente 0016-pe-2019 - o.d. 2 - ley de soli...,"MASSA, Sergio Tomás",AFIRMATIVO,https://votaciones.hcdn.gob.ar/votacion/3958
2,137-12-25,137,12,25,expediente 0016-pe-2019 - o.d. 2 - ley de soli...,"MASSA, Sergio Tomás",AFIRMATIVO,https://votaciones.hcdn.gob.ar/votacion/3957
3,137-12-24,137,12,24,expediente 0016-pe-2019 - o.d. 2 - ley de soli...,"MASSA, Sergio Tomás",AFIRMATIVO,https://votaciones.hcdn.gob.ar/votacion/3956
4,137-12-23,137,12,23,expediente 0016-pe-2019 - o.d. 2 - ley de soli...,"MASSA, Sergio Tomás",AFIRMATIVO,https://votaciones.hcdn.gob.ar/votacion/3955
5,137-12-22,137,12,22,expediente 0016-pe-2019 - o.d. 2 - ley de soli...,"MASSA, Sergio Tomás",AFIRMATIVO,https://votaciones.hcdn.gob.ar/votacion/3954
6,137-12-21,137,12,21,expediente 0016-pe-2019 - o.d. 2 - ley de soli...,"MASSA, Sergio Tomás",AFIRMATIVO,https://votaciones.hcdn.gob.ar/votacion/3953
7,137-12-20,137,12,20,expediente 0016-pe-2019 - o.d. 2 - ley de soli...,"MASSA, Sergio Tomás",AFIRMATIVO,https://votaciones.hcdn.gob.ar/votacion/3952
8,137-12-19,137,12,19,expediente 0016-pe-2019 - o.d. 2 - ley de soli...,"MASSA, Sergio Tomás",AFIRMATIVO,https://votaciones.hcdn.gob.ar/votacion/3951
9,137-12-18,137,12,18,expediente 0016-pe-2019 - o.d. 2 - ley de soli...,"MASSA, Sergio Tomás",AFIRMATIVO,https://votaciones.hcdn.gob.ar/votacion/3950


In [20]:
pd.DataFrame([x for x in votos]).rename(columns={'dip':'idDip'})

,idVotacion,idVoto,voto,numeroActa,idDip,dichos
0,137-8-13,10281,ausente,13,A51101,
1,137-8-13,10282,afirmativo,13,A4759,
2,137-8-13,10283,afirmativo,13,A50902,
3,137-8-13,10284,negativo,13,A51069,
4,137-8-13,10285,afirmativo,13,A51078,
...,...,...,...,...,...,...
252,137-8-13,10533,afirmativo,13,A51126,
253,137-8-13,10534,afirmativo,13,A51038,
254,137-8-13,10535,afirmativo,13,A51111,
255,137-8-13,10536,ausente,13,A50948,


In [22]:
pd.DataFrame([x for x in votos]).rename(columns={'dip':'idDip'}).merge(pd.DataFrame([x for x in diputados]), how='inner', on='idDip')

,idVotacion,idVoto,voto,numeroActa,idDip,dichos,urlPerf,fullName,apellido,nombre,estado,bloque,provincia,afirmativos,negativos,abstenciones,ausencias,presidencias
0,137-8-13,10281,ausente,13,A51101,,https://votaciones.hcdn.gob.ar/diputado/1675,"ZOTTOS, Miguel Andrés Costas",ZOTTOS,Miguel Andrés Costas,En actividad,Justicialista,Salta,418,161,52,366,-
1,137-8-13,10537,ausente,13,A51101,,https://votaciones.hcdn.gob.ar/diputado/1675,"ZOTTOS, Miguel Andrés Costas",ZOTTOS,Miguel Andrés Costas,En actividad,Justicialista,Salta,418,161,52,366,-
2,137-8-13,10282,afirmativo,13,A4759,,https://votaciones.hcdn.gob.ar/diputado/395,"ABDALA DE MATARAZZO, Norma Amanda",ABDALA DE MATARAZZO,Norma Amanda,En actividad,Frente de Todos,Santiago del Estero,741,53,5,155,102
3,137-8-13,10283,afirmativo,13,A50902,,https://votaciones.hcdn.gob.ar/diputado/572,"ACERENZA, Samanta María Celeste",ACERENZA,Samanta María Celeste,-,PRO,Buenos Aires,340,91,3,86,-
4,137-8-13,10284,negativo,13,A51069,,https://votaciones.hcdn.gob.ar/diputado/1645,"AICEGA, Juan",AICEGA,Juan,En actividad,PRO,Buenos Aires,140,91,-,9,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,137-8-13,10532,afirmativo,13,A51095,,https://votaciones.hcdn.gob.ar/diputado/1669,"YASKY, Hugo",YASKY,Hugo,En actividad,Frente de Todos,Buenos Aires,123,49,2,66,-
253,137-8-13,10533,afirmativo,13,A51126,,https://votaciones.hcdn.gob.ar/diputado/1700,"YEDLIN, Pablo Raúl",YEDLIN,Pablo Raúl,En actividad,Frente de Todos,Tucumán,200,17,5,18,-
254,137-8-13,10534,afirmativo,13,A51038,,https://votaciones.hcdn.gob.ar/diputado/1610,"ZAMARBIDE, Federico Raúl",ZAMARBIDE,Federico Raúl,En actividad,Unión Cívica Radical,Mendoza,141,86,1,12,-
255,137-8-13,10535,afirmativo,13,A51111,,https://votaciones.hcdn.gob.ar/diputado/1685,"ZAMORA, Claudia",ZAMORA,Claudia,-,Frente Cívico por Santiago,Santiago del Estero,101,15,1,96,-


In [24]:
pd.DataFrame([x for x in votos]).rename(columns={'dip':'idDip'}).merge(pd.DataFrame([x for x in diputados]), how='inner', on='idDip').merge(pd.DataFrame([x for x in actas]).rename(columns={'idActa':'idVotacion'}), how='inner', on='idVotacion').columns

Index(['idVotacion', 'idVoto', 'voto', 'numeroActa_x', 'idDip', 'dichos',
       'urlPerf', 'fullName', 'apellido', 'nombre', 'estado', 'bloque',
       'provincia', 'afirmativos', 'negativos', 'abstenciones', 'ausencias',
       'presidencias', 'periodo', 'reunion', 'numeroActa_y', 'titulo',
       'presidente', 'resolucion', 'urlActa'],
      dtype='object')